In [3]:
import torch
import torch.nn.functional as F
from torch import nn  # for

import numpy as np  # for
import pandas as pd  # for


import plotly.graph_objects as go
import plotly.express as px
from plotly import subplots


In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("cuda available")
else:
    device = torch.device("cpu")

<div dir="rtl" lang="he" xml:lang="he">


# מודלים יוצרים - _gan models_
ניתן להשתמש ברשתות נויירונים גם ליצירה של מידע הנראה כאמיתי. 
נבנה מודל כזה שיוכל ליצור כתב יד בצורה אמינה.

<div dir="rtl" lang="he" xml:lang="he">

### יבוא הדאטא
נוריד את הקבצים מאתר האינטרנט בצורה ידנית, ונלחץ אותם במקום הנכון.
הדאטא מגיע כקובץ CSV טבלה, כך שהעמודה הראשונה היא התגית _label_ וכל שאר העמודות הם העמודות של התמונה


נשתמש בפונקצייה `load_text` בשביל לטעון את הקבצים, 
לאחר מכן נמיר את הקבצים לוקטור.

In [ ]:
#script to download files 
##!!! only for files that downloaded at `.tar.gz` format


import requests         #for downloading files
import tarfile          #for extracting files
import pathlib          #check if file exists

URL = "https://www.kaggle.com/datasets/hojjatk/mnist-dataset/download?datasetVersionNumber=1"
DATA_FOLDER = "data"
MODEL_NAME = "mnist"
data_folder = pathlib.Path(f"./{DATA_FOLDER}")
data_set_folder = pathlib.Path(f"{DATA_FOLDER}/{MODEL_NAME}")
downloaded_file =  pathlib.Path(f"{DATA_FOLDER}/{MODEL_NAME}/{MODEL_NAME}.tar.gz")


if not data_folder.exists():
    data_folder.mkdir()
if not data_set_folder.exists():
    data_set_folder.mkdir()
    response = requests.get(URL)
    open(downloaded_file, "wb").write(response.content)
if downloaded_file.exists():
    # extracting file
    file = tarfile.open(downloaded_file)
    file.extractall(data_set_folder)
    file.close()
    #delet downloaded file
    if input("do you want to delete the downloaded file?\ny or n") in "Yy":
        downloaded_file.unlink()



In [5]:
vec = np.genfromtxt(
    "data/mnist/mnist_test/mnist_test.csv", delimiter=",", skip_header=1
)
print(
    f"""
vec shape: {vec.shape}
vec dtpye: {vec.dtype}
vec head: 
{vec[0:5]}
    """
)


vec shape: (10000, 785)
vec dtpye: float64
vec head: 
[[7. 0. 0. ... 0. 0. 0.]
 [2. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]]
    


<div dir="rtl" lang="he" xml:lang="he">

העמודה הראשונה היא התוית _label_ 
של התמונה, וכל שאר 784 העמודות זה התמונה בשחור לבן. 

In [6]:
labels = torch.from_numpy(vec[:,0]).reshape(len(vec),1).to(dtype=torch.float32)
data = torch.from_numpy(vec[:,1:]).reshape(10000,28,28).to(dtype=torch.float32)
data_set = (data,labels)
print(labels)

tensor([[7.],
        [2.],
        [1.],
        ...,
        [4.],
        [5.],
        [6.]])


<div dir="rtl" lang="he" xml:lang="he">


### הצגת המספרים
בפונקציה  
`imshow` 
של 
_plotly_ 
ניתן להציג סדרת תמונות


In [7]:
fig = px.imshow(data_set[0][0:10],facet_col=0,facet_col_wrap=4,labels={'facet_col':'index'})
fig.show()
print(data_set[1][0:10])

tensor([[7.],
        [2.],
        [1.],
        [0.],
        [4.],
        [1.],
        [4.],
        [9.],
        [5.],
        [9.]])


<div dir="rtl" lang="he" xml:lang="he">

## הרשת השופטת
נבנה את המודל שיעריך האם התמונות שניתנות לו הם אמיתיות או לא


מבנה הרשת הוא פשוט: 
יש שלוש שכבות ליניאריות 
_dence_
עם פונקציית אקטיבציה 
_ReLU_
לאחר כל שכבה כזאת מורידים חלק מהוקטורים עם 
_dropout_ 

שפר לשים לב שהשכבותהלינאירות בהתחלה מגדילים את התמונה ורק לאחר מקטינים ומחליטים מה תהיה התמונה. 

In [8]:
class Discriminator(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = x.view(x.size(0), 784)
        output = self.model(x)
        return output

In [9]:
discriminator = Discriminator()
x = data_set[0][4:10]
discriminator(x)

tensor([[0.1325],
        [0.0894],
        [0.2759],
        [0.6371],
        [0.9874],
        [0.1003]], grad_fn=<SigmoidBackward0>)

<div dir="rtl" lang="he" xml:lang="he">

## הגנרטור 
הארכיטקטורה של הגנרטור היא בדיוק ההפוכה לארכיטקטורה של המחליט 
_discriminator_ /
ההבדל בן השכבות הוא בפונקציית האקטיבציה, 
_sigmoid_ 
מול 
_tahn_
ובשכבות ה
_dropout_ 
שאותן אין טעם לשים במודל הגנרטיבי


אפשר לראות שהאברים שהוא מקבל הם בגודל 100.

In [10]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 784),
            nn.Tanh(),
        )

    def forward(self, x):
        output = self.model(x)
        output = output.view(x.size(0), 28, 28)
        return output


In [12]:
generator = Generator()
x = torch.randn(10,100)
px.imshow(generator(x).detach(),facet_col=0,facet_col_wrap=4)

<div dir="rtl" lang="he" xml:lang="he">

# אימון המודל
נבנה  `Trainer` בשביל לאמן את המודלים שלנו
הוא יכיל: 
- שני מודלים , את המודל היוצר 
`Generator`
ואת המודל המבקר
`Discriminator`
- לכל אחד מהמודלים נגדיר אופטימייזר שונה

### תהליך האימון 

לכל קבוצהFIXIT נעשה את הדברים הבאים:
ניצור קבוצת תמונות פייק (מתוך התפלגות נורמלית) ונאמן את המודל השופטFIXIT
להבחין בן התמונות. 
- ניתן לגנרטור ליצור כמה תמונות ונראה אם האלגוריתם השופט מזהה שהם לא תמונות אמיתיות

In [13]:
from typing import Union, Type, Callable,Optional,Any,List,Tuple

ModelType = nn.Module
TensorType = torch.Tensor
OptimazerType = Type[torch.optim.Optimizer]
LossFunctoinType = Callable[[TensorType,TensorType],TensorType]
SchedulerType = Type[torch.optim.lr_scheduler.LRScheduler]
VisulizeFuntoinType = Callable[[float,Optional[Any]],None]
DataType = Tuple[torch.Tensor,torch.Tensor]
PathType = str
FileType = str


In [20]:
import math  # for cilo
from tqdm.notebook import tqdm

DATA = 0 
LABELS = 1
class TrainerGan:
    def __init__(
        self,
        discriminator: ModelType,
        generator: ModelType,
        loss_function: LossFunctoinType,
        optimizer_discriminator: OptimazerType,
        optimizer_generator:OptimazerType,
        scheduler: Optional[SchedulerType]=None,
    ):
        self.discriminator = discriminator
        self.generator = generator
        self.loss_function = loss_function
        self.optimizer_discriminator = optimizer_discriminator
        self.optimizer_generator = optimizer_generator
        self.scheduler = scheduler
        self.loss_list_D = []
        self.loss_list_G = []
        self.data = None

    def train(
        self, data_set: DataType, batch_size: int, times: int = 1, visualization: int = 10
    ):
        #creat figure to disply the loss funtion
        num_iteration = math.ceil(len(data_set[0]) / batch_size)  # round up number of iteration
        fig = go.Figure(
            data=go.Scatter(x=torch.arange(0, num_iteration), y=self.loss_list_D)
        ).update_yaxes(range=[0, 1])

        
        for epoch in range(times):
            for i in tqdm(range(num_iteration)):


                # Data for training the discriminator
                real_samples = data_set[DATA][i*batch_size:(i+1)*batch_size]
                real_samples_labels = data_set[LABELS][i*batch_size:(i+1)*batch_size]  
                latent_space_samples = torch.randn(batch_size,100)


                #generate numbers
                self.generator.eval()
                generated_samples = self.generator.forward(latent_space_samples)
                generated_labels = torch.zeros(batch_size,1)
                all_samples = torch.cat((real_samples,generated_samples))
                all_labels = torch.cat((real_samples_labels,generated_labels))

                # Training the discriminator
                self.discriminator.train()
                self.discriminator.zero_grad()
                output_discriminator = self.discriminator.forward(all_samples)
                loss_D = self.loss_function(output_discriminator,all_labels)
                loss_D.backward()
                self.optimizer_discriminator.step()

                # Data for training the generator
                latent_space_samples = torch.randn((batch_size, 100))

                self.generator.train()
                self.generator.zero_grad()
                generated_samples = self.generator(latent_space_samples)
                self.discriminator.eval()
                output_discriminator_generated = self.discriminator(generated_samples)
                loss_G = self.loss_function(
                    output_discriminator_generated, torch.ones(batch_size,1)
                )
                loss_G.backward()
                self.optimizer_generator.step()

        


```python
trainer = Triner(model,loss,optim,sch,visul)
trainer.set_plain("first plain", )
trainer.train(data,10,shappe=Treu,lr=0.1,gamma:float=None,schler_gama:str="",plean:str="")
```

In [21]:
discriminator = Discriminator()
generator = Generator()

In [22]:
lr = 0.0001
num_epochs = 50
loss_function = nn.BCELoss()

optimizer_discriminator = torch.optim.Adam(discriminator.parameters(), lr=lr)
optimizer_generator = torch.optim.Adam(generator.parameters(), lr=lr)

In [23]:
trainer = TrainerGan(discriminator,generator,loss_function,optimizer_discriminator,optimizer_generator)

In [24]:
x = torch.randn(10,100)
generator.eval()
all_out = generator(x)

In [26]:
from winsound import Beep
from IPython.display import clear_output
for _ in range(20):
    trainer.train(data_set,2)
    x = torch.randn(10,100)
    generator.eval()
    out = generator(x)
    all_out = torch.cat((all_out,out))
    im = all_out.detach().reshape(-1,10,28,28)
    clear_output(wait=True)
    px.imshow(im,animation_frame=0,facet_col=1,facet_col_wrap=4,labels={"animation_frame":"round"}).show()
    Beep(400, 250)

  0%|          | 0/5000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [88]:
import pathlib
DATA_FOLDER = "data"
MODEL_NAME = "mnist"

folder_path = pathlib.Path(f"{DATA_FOLDER}/{MODEL_NAME}/trained_models")
model_path =  pathlib.Path(f"{DATA_FOLDER}/{MODEL_NAME}/trained_models/{MODEL_NAME}_gan_G_0604.pth")
if input("do you want save module on disc?") in "yY":
    if not folder_path.exists():
        folder_path.mkdir()

    torch.save(generator.state_dict(), model_path)

In [2]:
from tqdm.notebook import tqdm
import time

a = str("abcdefghijklmnopqrstuvwxyz")
for i in tqdm(list(a)):
    time.sleep(0.1)
    print(i,end=" ")

  0%|          | 0/26 [00:00<?, ?it/s]

a b c d e f g h i j k l m n o p q r s t u v w x y z 